<a href="https://colab.research.google.com/github/kangdoung/CS114.K21/blob/master/Sarcasm_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nhóm :
* Dương Lê Tường Khang - 18520882
* Bùi Đào Gia Huy - 18520818
* Lã Trường Hải - 18520698

In [94]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"khangdlt","key":"3dc1f86a2ffc9a36ce11a32beed2db24"}'}

Upload file chứa key của kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Cài đặt môi trường

In [3]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection


  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


Dowload dataset từ kaggle về.

In [4]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


Giải nén dataset thành 2 file 
* Sarcasm_Headlines_Dataset.json 
* Sarcasm_Headlines_Dataset_v2.json

---



# Dataset của bài toán.

## Mô tả dataset


### **1. Mô tả bài toán**:
&ensp;&ensp;Phân loại 1 tựa đề của bài báo là *châm biếm (sarcastic)* hay *không châm biếm (non-sarcastic)*.
### **2. Cách thu thập dữ liệu train**:
Tiêu đề các bài báo được thu thập từ 2 trang:
* HuffPost: các tiêu đề được thu thập từ trang báo này sẽ được dán nhãn 0 ( non-sarcastic).
* TheOnion: nhãn 1 (sarcastic) sẽ được dán cho những tiêu đề được thu thập.Nhãn được thu thập ở 2 mục Brief và Photos Categories


## Đọc dataset

In [5]:
import pandas as pd
f1=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
f2=pd.read_json('Sarcasm_Headlines_Dataset_v2.json',lines=True)

Đọc file json từ thư viện pandas

In [52]:
f1.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [53]:
f2.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
f2

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...


In [51]:
df=pd.concat([f1,f2],axis=0,sort=False)
df.describe()

,is_sarcastic
count,55328.000000
mean,0.458321
std,0.498264
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


Nhóm 2 file Sarcasm_Headlines_Dataset.json và Sarcasm_Headlines_Dataset_v2.json chung với nhau

In [7]:
df=df.drop('article_link',axis=1)
df

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
28614,jews to celebrate rosh hashasha or something,1
28615,internal affairs investigator disappointed con...,1
28616,the most beautiful acceptance speech this week...,0
28617,mars probe destroyed by orbiting spielberg-gat...,1


Loại bỏ đi cột "article_link" do trong quá trình xử lý không cần đến.

# Feature Engineering

In [20]:
X=df['headline']
y=df['is_sarcastic']

Lựa chọn feature


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['headline'],df['is_sarcastic'],test_size=0.2,random_state=101)

Chia thành 2 tập test và train

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [54]:
from sklearn.preprocessing import StandardScaler

Cài đặt các thư viện cần thiết

In [72]:
pipe = Pipeline([('vect',CountVectorizer()),
               ('tfidf',TfidfTransformer()),
               ('model',LogisticRegression())])

model = pipe.fit(X_train, y_train)

Thay vì làm các công đoạn riêng, pipeline cho phép gộp các công đoạn lại với nhau.
* Counvectorizer: chuyển text thành vector
* TFidfTransformer: tính giá trị idf cho ma trận trên
* LogisticRegression: model lựa chọn để train

Có thể thêm nhiều bước trung gian

Khi model gọi fit thì các object 'vect' và 'tfidf' sẽ tiến hành gọi 2 method fit và transform. Và ở bước cuối cùng thì object 'model' sẽ gọi method fit để tối ưu các tham số.

In [73]:
pred=model.predict(X_test)
print(accuracy_score(y_test,pred))

0.8942707392011567


Đánh giá của model dựa trên accuracy

Accuracy là độ đo được đánh giá bằng cách (số câu dự đoán đúng) / (toàn bộ câu của văn bản)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90      5989
           1       0.89      0.88      0.88      5077

    accuracy                           0.89     11066
   macro avg       0.89      0.89      0.89     11066
weighted avg       0.89      0.89      0.89     11066



## Sarcasm detect  1 headline bất kì

In [65]:
import numpy as np

In [92]:
hdlines = input()
hdline_pre = [hdlines]
flag = model.predict(hdline_pre)
if flag == 1:
  print("sarcastic")
else:
  print("non-sarcastic")

Civil Whites
is sarcastic


# Predict trên 2000 dữ liệu crawl về từ 2 trang web như tập dataset trên.

## Cách thu thập dữ liệu

## Dữ liệu thu thập


In [95]:
import pandas as pd
df1 = pd.read_csv("data_crawl.csv")
df1

,Article,Headlines,is_sarcasm
0,https://www.huffpost.com/entry/unemployment-nu...,15 Million More Americans Sought Unemployment ...,0
1,https://ogn.theonion.com/10-clumps-of-grass-in...,10 Clumps Of Grass In Video Games,1
2,https://ogn.theonion.com/10-clumps-of-grass-in...,10 Clumps Of Grass In Video Games,1
3,https://ogn.theonion.com/10-clumps-of-grass-in...,10 Clumps Of Grass In Video Games,1
4,https://ogn.theonion.com/10-clumps-of-grass-in...,10 Clumps Of Grass In Video Games,1
...,...,...,...
1995,https://local.theonion.com/zoom-crasher-become...,Zoom Crasher Becomes Too Engrossed In Sales Me...,1
1996,https://www.huffpost.com/entry/tik-tok-kpop-zo...,Zoomers Boast They Sabotaged Trump Rally Turno...,0
1997,https://www.huffpost.com/entry/zuckerberg-revi...,Zuckerberg Says Hell Review Policies That Allo...,0
1998,https://www.huffpost.com/entry/zuckerberg-revi...,Zuckerberg Says Hell Review Policies That Allo...,0


Dữ liệu mới thu thập

In [97]:
df1=df1.drop('Article',axis=1)


In [98]:
df1

,Headlines,is_sarcasm
0,15 Million More Americans Sought Unemployment ...,0
1,10 Clumps Of Grass In Video Games,1
2,10 Clumps Of Grass In Video Games,1
3,10 Clumps Of Grass In Video Games,1
4,10 Clumps Of Grass In Video Games,1
...,...,...
1995,Zoom Crasher Becomes Too Engrossed In Sales Me...,1
1996,Zoomers Boast They Sabotaged Trump Rally Turno...,0
1997,Zuckerberg Says Hell Review Policies That Allo...,0
1998,Zuckerberg Says Hell Review Policies That Allo...,0


Tương tự loại bỏ cột Link.

In [99]:
df1.columns

Index(['Headlines', 'is_sarcasm'], dtype='object')

Các cột có trong dữ liệu

### Feature engineering

In [96]:
nX=df1['Headlines']
ny=df1['is_sarcasm']
ny

0       0
1       1
2       1
3       1
4       1
       ..
1995    1
1996    0
1997    0
1998    0
1999    0
Name: is_sarcasm, Length: 2000, dtype: int64

Tương tự chọn headline làm feature và chọn is_sarcasm làm label

# Logistic Regression

In [100]:
n_model = pipe.fit(X,y)

Train lại model trên với tập dữ liệu ban đầu

In [101]:
pred = n_model.predict(nX)

In [102]:
accuracy_score(ny, pred)

0.7135

Kết quả thấp so với kết quả của model ban đầu.


# Multinomial Naive Bayes

In [103]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [104]:
from sklearn.naive_bayes import MultinomialNB

In [105]:
pl = Pipeline([
    ('bow', CountVectorizer()),
    ('tfidf', TfidfTransformer()),  
    ('classifier', MultinomialNB()),  
])

In [106]:
model = pl.fit(X,y)

In [ ]:
model

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [107]:
predict = model.predict(nX)

In [108]:
accuracy_score(ny, predict)

0.708

Kết quả cho ra tốt hơn model Logistic Regression.